In [2]:
import os
os.getcwd()
os.chdir(r'C:\Users\windows\Downloads')

In [3]:
import tensorflow as  tf
from keras.preprocessing.image import ImageDataGenerator


In [18]:
## Preprocessing the Training Set 
train_datagen = ImageDataGenerator(
    rescale =1./255, ## Feature scaling to get individual pixels 
    shear_range = 0.2, # Geometric transformations to reduce overfitting 
    zoom_range = 0.2,  # "" ^
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    'chest_xray/train', 
    target_size = (64,64),
    batch_size = 32, 
    class_mode = 'binary')


Found 5214 images belonging to 2 classes.


In [19]:
## Preprocessing the test set 

test_datagen = ImageDataGenerator(1./255) 
test_set = test_datagen.flow_from_directory(
    'chest_xray/test', 
    target_size = (64,64),
    batch_size = 32, 
    class_mode = 'binary')

Found 624 images belonging to 2 classes.


In [34]:
## Building the CNN 

cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu', input_shape=[64,64,3])) ## Change the filter if needed to investigate different results

## Pooling 

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## 2nd Layer 
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu')) ## Change the filter if needed to investigate different results
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Flattening 

cnn.add(tf.keras.layers.Flatten())

## Full Connection 

cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu')) ## More units might yield better results 

## Output Layer 

cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) ## Binary classification = sigmoid activation, 1 neuron in output layer




In [35]:
## Training the CNN 
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=[ 'accuracy' ])

In [36]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 27)

Epoch 1/27
163/163 [==============================] - 47s 288ms/step - loss: 0.3840 - accuracy: 0.8299 - val_loss: 51.1300 - val_accuracy: 0.8221
Epoch 2/27
163/163 [==============================] - 46s 285ms/step - loss: 0.2408 - accuracy: 0.8966 - val_loss: 50.8230 - val_accuracy: 0.8446
Epoch 3/27
163/163 [==============================] - 46s 279ms/step - loss: 0.2054 - accuracy: 0.9145 - val_loss: 46.9545 - val_accuracy: 0.8814
Epoch 4/27
163/163 [==============================] - 46s 280ms/step - loss: 0.1862 - accuracy: 0.9271 - val_loss: 79.4869 - val_accuracy: 0.8494
Epoch 5/27
163/163 [==============================] - 46s 283ms/step - loss: 0.1585 - accuracy: 0.9392 - val_loss: 48.2169 - val_accuracy: 0.8798
Epoch 6/27
163/163 [==============================] - 46s 280ms/step - loss: 0.1660 - accuracy: 0.9379 - val_loss: 50.8882 - val_accuracy: 0.8798
Epoch 7/27
163/163 [==============================] - 47s 286ms/step - loss: 0.1470 - accuracy: 0.9425 - val_loss: 57.0109 -

In [23]:
## Making a single Prediction 

import numpy as np 
from keras.preprocessing import image 

test_image = tf.keras.preprocessing.image.load_img('chest_xray/single_prediction/person1_bacteria_1.jpeg', target_size = (64,64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
training_set.class_indices ## Tells us which index corresponds to what 'value'
if result[0][0] == 0:
    prediction = 'This patient is healthy'
else: 
    prediction = 'This patient has pneumonia'

1/1 [==============================] - 0s 65ms/step


In [24]:
print(prediction)

This patient has pneumonia


In [28]:
m = tf.keras.metrics.FalseNegatives()
m.result().numpy()

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              